<a href="https://colab.research.google.com/github/amithrx/amithrx-Bag-of-words-based-matching-on-the-MNIST-fashion-datasets/blob/main/Bag_of_Words.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2
import math
print(cv2.__version__)

4.6.0


In [ ]:
fashion_mnist = tf.keras.datasets.fashion_mnist
(trainImageData,trainImageLabel), (testImageData,testImageLabel) = fashion_mnist.load_data()

In [ ]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

In [ ]:
def Get_features_set(ImageData):
  '''
  Returns feature vector containing descriptors corresponding to given image vector
  '''
  sift = cv2.xfeatures2d.SIFT_create()
  imageDescriptor = []
  length = len(ImageData)
  for i in range(length):
    img = ImageData[i]
    image8bit = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX).astype('uint8')
    k,d = sift.detectAndCompute(image8bit,None)
    if d is None: imageDescriptor.append([])
    else : imageDescriptor.append(list(d))
  return imageDescriptor

In [ ]:
def Merge_features(descriptorSet):
  '''
  To combine all the features in given descriptorSet to a single Descriptor array
  '''
  Descriptors = []
  length = len(descriptorSet)
  for i in range(length):
    descriptorLength = len(descriptorSet[i])
    for j in range(descriptorLength):
      Descriptors.append(descriptorSet[i][j])
  return np.array(Descriptors)

In [ ]:
def Centroid(cluster): 
  '''
  Returns the centroid of a particular cluster
  '''
  cluster = np.array(cluster)
  return np.mean(cluster,axis = 0)


In [ ]:
def Nearest_centroid(data,centroids):
  '''
  Returns the index of the nearest centroid of a feature descriptor 
  '''
  k = len(centroids)
  differenceArray = []
  for i in range(k):
    differenceArray.append(np.linalg.norm(data-centroids[i]))
  differenceArray = np.array(differenceArray)
  return int(differenceArray.argmin())

In [ ]:
def Initialize_centroid(k,descriptors):
  '''
      Initialising k centroids of the feature descriptor
      Used KMeans++ algorithm for initializing the centroid,
      where in next iteration we will choose randomly the points with probability assigned to each descriptors
  '''
  n = len(descriptors)
  x = list(np.random.choice(n,size = 1,replace = False))
  ret = []
  ret.append(descriptors[x[0],::])
  for i in range(k-1):
    D = [0]*n
    W = [0]*n
    total = 0
    for j in range(n):
      nearest = ret[Nearest_centroid(descriptors[j,::],ret)]
      D[j] = np.linalg.norm(descriptors[j,::]-nearest)
      total += D[j]**2
    for j in range(n):
      W[j] = (D[j]**2)/total
    x = list(np.random.choice(n,size = 1,replace = False,p = W))
    ret.append(descriptors[x[0],::])
  #   print(ret)
  return np.array(ret)

In [ ]:
def kMean_clustering(k,centroids,descriptors):
  '''
  Running Single Iteration of a K-mean clustering
  '''
  n = len(descriptors)
  color = [0]*n
  for i in range(n):
    color[i] = Nearest_centroid(descriptors[i],centroids)    
  newCentroids = []
  cluster = {}
  for i in range(k): cluster[i] = []
  for j in range(n): cluster[color[j]].append(descriptors[j])
  for i in range(k):
    newCentroids.append(Centroid(cluster[i]))
  return (color,np.array(newCentroids))  

In [ ]:
def Get_true_descriptor(centroids,descriptors):
  '''
  Getting the nearest descriptor to each centroids obtained 
  '''
  k = len(centroids)
  realRep = []
  for i in range(k):
    realRep.append(descriptors[Nearest_centroid(centroids[i],descriptors)])
  return realRep

In [ ]:
def ComputeHistogram(clusterCentroids,imageDescriptor):
  '''
  Creating Histogram for every images in sets
  '''
  Histogram=[]
  length = len(imageDescriptor)
  for i in range(length):
    descriptors = imageDescriptor[i]
    k = len(clusterCentroids)
    descriptorNum=len(descriptors)
    frequencyVector=np.zeros(k)
    for j in range(descriptorNum):
      frequencyVector[Nearest_centroid(descriptors[j],clusterCentroids)]+=1
    Histogram.append(frequencyVector)
  return np.array(Histogram)

In [ ]:
def Count_image_containing_visual_word(histogramSet):
  '''
  Storing total count of images which contain visual word i
  '''
  k = histogramSet.shape[1]
  count = np.zeros(k)
  for histogram in histogramSet:
    for i in range(k):
      if histogram[i] > 0: count[i]+=1
  return count

In [ ]:
def Normalize_histogram(histogramSet,countImages):
  '''
  Normalizing the histogram, by assigning more weight to particular word as compared to other weight 
  by using TF-IDF methods
  '''
  newHistogramSet = []
  for histogram in histogramSet:
    nd = 0
    N = len(histogramSet)
    k = len(histogram)
    for i in range(k):
      if histogram[i] > 0: nd+=1
    for i in range(k):
      if nd == 0 or countImages[i] == 0: histogram[i] = 0
      else :histogram[i] = (histogram[i]/nd)*(math.log(N/countImages[i]))
    newHistogramSet.append(histogram)
  return np.array(newHistogramSet)

In [ ]:
def MatchHistogram(A,B): 
  '''
  Computing the cosine similarity of two images using their histogram
  '''
  a = np.linalg.norm(A)
  b = np.linalg.norm(B)
  if a == 0:
    if b == 0: return 0
    else: return 1
  elif b == 0: return 1
  return 1 - (np.dot(A,B))/(a)/(b)

In [ ]:
def Accuracy(trainImageLabels,testImageLabels,trainHistogramSet,testHistogramSet,kk):
  '''
  Getting accuracy
  '''
  n = len(trainImageLabels)
  m = len(testImageLabels)
  # m=100
  predicted = [0]*m
  for i in range(m):
    freq = [0]*10
    freq = np.array(freq)
    differenceArray = []
    print("Image" + str(i) + " test")
    for j in range(n):
      differenceArray.append(MatchHistogram(testHistogramSet[i],trainHistogramSet[j]))
    differenceArray = np.array(differenceArray)
    maxIndices = np.argsort(differenceArray)
    for j in range(kk):
      freq[trainImageLabels[maxIndices[j]]] += 1
    x = freq.argmax()
    for j in range(10):
      if freq[x] == freq[j]:
        predicted[i]=j

  return predicted

In [ ]:
# def accuracy2(trainImageLabels,testImageLabels,trainHistogramSet,testHistogramSet,kk):
#   n = len(trainImageLabels)
#   m = len(testImageLabels)
#   # m=100
#   predicted = [0]*m
#   positive = 0
#   classwise=[0]*10
#   total_classwise = [0]*10
#   true_positive = [0]*10
#   false_positive = [0]*10
#   false_negative = [0]*10
#   precision = [0]*10
#   recall = [0]*10
#   for i in range(m):
#     total_classwise[testImageLabels[i]] += 1
#     freq = [0]*10
#     freq = np.array(freq)
#     differenceArray = []
#     print("Image" + str(i) + " test")
#     for j in range(n):
#       differenceArray.append(MatchHistogram(testHistogramSet[i],trainHistogramSet[j]))
#     differenceArray = np.array(differenceArray)
#     maxIndices = np.argsort(differenceArray)
#     for j in range(kk):
#       freq[trainImageLabels[maxIndices[j]]] += 1
#     x = freq.argmax()
#     for j in range(10):
#       if freq[x] == freq[j]:
#         predicted[i]=j
#         if testImageLabels[i] == j:
#           positive +=1
#           true_positive[j] +=1
#           classwise[j] +=1
#           break
#         else:
#           false_positive[j] +=1
#       else:
#         if testImageLabels[i] ==j:
#           false_negative[j] +=1

#   for j in range(10):
#     classwise[j]=(classwise[j]/total_classwise[j])*100
#     precision[j]=(true_positive[j])/(true_positive[j]+false_positive[j])
#     recall[j]=(true_positive[j])/(true_positive[j]+false_negative[j])
#     # print(positive)
#   total_accuracy = (positive/m)*100
#   return (total_accuracy,classwise,precision,recall,predicted)

In [ ]:
def CreateVisualDictionary(k,trainDescriptors):
  print("On k = " + str(k))
  centroids = Initialize_centroid(k,trainDescriptors)
  n = len(trainDescriptors)
  clusterCentroid = [0]*n
  for it in range(20):
      print("Iteration " + str(it) + ":")
      (clusterCentroid,centroids) = kMean_clustering(k,centroids,trainDescriptors)
  centroids = Get_true_descriptor(centroids,trainDescriptors)
  np.savetxt('./kmeanCentoids15.txt',centroids)
  return centroids


In [ ]:
print("Extracting Testing Data..")
testImageDescriptor = Get_features_set(testImageData)
testDescriptors = Merge_features(testImageDescriptor)
print("Extracting Training Data..")
trainImageDescriptor = Get_features_set(trainImageData)
trainDescriptors = Merge_features(trainImageDescriptor)

print("Creating Visual Dictionary..")
k = 15 #Explained in a pdf file
centroids=CreateVisualDictionary(k,trainDescriptors)

print("Forming histograms of training set..")
trainHistogramSet = ComputeHistogram(centroids,trainImageDescriptor)
print("Forming histograms of testing set..")
testHistogramSet = ComputeHistogram(centroids,testImageDescriptor)

countImagesTrain = Count_image_containing_visual_word(trainHistogramSet)
countImagesTest = Count_image_containing_visual_word(testHistogramSet)

print("Normalizing histograms of training set..")
trainHistogramSet = Normalize_histogram(trainHistogramSet,countImagesTrain)
print("Normalizing histograms of testing set..")
testHistogramSet = Normalize_histogram(testHistogramSet,countImagesTest)


Extracting Testing Data..
Extracting Training Data..
Creating Visual Dictionary..
On k = 15
Iteration 0:
Iteration 1:
Iteration 2:
Iteration 3:
Iteration 4:
Iteration 5:
Iteration 6:
Iteration 7:
Iteration 8:
Iteration 9:
Iteration 10:
Iteration 11:
Iteration 12:
Iteration 13:
Iteration 14:
Iteration 15:
Iteration 16:
Iteration 17:
Iteration 18:
Iteration 19:
Forming histograms of training set..
Forming histograms of testing set..
Normalizing histograms of training set..
Normalizing histograms of testing set..


In [ ]:
print("Testing image sets..")
predicted = Accuracy(trainImageLabel,testImageLabel,trainHistogramSet,testHistogramSet,15)

from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
y_true = testImageLabel
y_predict = predicted
print(classification_report(y_true, y_predict, target_names=class_names))
print("Overall accuracy => ")
print(accuracy_score(y_true, y_predict))
# print("Overall classification accuracy is: "+str(total_accuracy))
# for i in range(10):
#   print("Class "+str(i)+" has accuracy: ",+(classwise[i]))
#   print("Class "+str(i)+" has precision: ",+(precision[i]))
#   print("Class "+str(i)+" has recall: ",+(recall[i]))

Testing image sets..
Image0 test
Image1 test
Image2 test
Image3 test
Image4 test
Image5 test
Image6 test
Image7 test
Image8 test
Image9 test
Image10 test
Image11 test
Image12 test
Image13 test
Image14 test
Image15 test
Image16 test
Image17 test
Image18 test
Image19 test
Image20 test
Image21 test
Image22 test
Image23 test
Image24 test
Image25 test
Image26 test
Image27 test
Image28 test
Image29 test
Image30 test
Image31 test
Image32 test
Image33 test
Image34 test
Image35 test
Image36 test
Image37 test
Image38 test
Image39 test
Image40 test
Image41 test
Image42 test
Image43 test
Image44 test
Image45 test
Image46 test
Image47 test
Image48 test
Image49 test
Image50 test
Image51 test
Image52 test
Image53 test
Image54 test
Image55 test
Image56 test
Image57 test
Image58 test
Image59 test
Image60 test
Image61 test
Image62 test
Image63 test
Image64 test
Image65 test
Image66 test
Image67 test
Image68 test
Image69 test
Image70 test
Image71 test
Image72 test
Image73 test
Image74 test
Image75 test
I